<a href="https://colab.research.google.com/github/pavansai26/convert-the-models-into-c-file-for-deployment-in-micro-controllers/blob/master/convert_the_models_into_c_file_for_deployment_in_micro_controllers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math


In [0]:
samples=1000
np.random.seed(1337)
x_values=np.random.uniform(low=0,high=2*math.pi,size=samples)
np.random.shuffle(x_values)
y_values=np.sin(x_values)
plt.plot(x_values,y_values)

In [0]:
# add random noise
y_values+=0.1*np.random.randn(*y_values.shape)
plt.plot(x_values,y_values)

In [0]:
train_split=int(0.6*samples)
test_split=int(0.2*samples+train_split)

In [0]:
x_train,x_test,x_validate=np.split(x_values,[train_split,test_split])

In [0]:
y_train,y_test,y_validate=np.split(y_values,[train_split,test_split])

In [0]:
#double check that our splits add up correctly
assert(x_train.size+x_test.size+x_validate.size)==samples

In [0]:
# plot the data in each partition in different colors
plt.plot(x_train,y_train,label='train')
plt.plot(x_test,y_test,label='test')
plt.plot(x_validate,y_validate,label='validate')
plt.legend()
plt.show()

In [0]:
import tensorflow as tf
from tensorflow.keras import layers


In [0]:
model_1=tf.keras.Sequential()
model_1.add(layers.Dense(16,activation='relu',input_shape=(1,)))
model_1.add(layers.Dense(1))

In [0]:
model_1.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])

In [0]:
history_1=model_1.fit(x_train,y_train,epochs=1000,batch_size=64,validation_data=(x_validate,y_validate))

In [0]:
history_1=model_1.fit(x_train,y_train,epochs=1000,validation_data=(x_validate,y_validate))

In [0]:
loss=history_1.history['loss']
val_loss=history_1.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'g.',label='training loss')
plt.plot(epochs,val_loss,'b',label='validation loss')
plt.title('training and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [0]:
# exclude the first few epochs so the graph will be easier to read
skip=50
plt.plot(epochs[skip:],loss[skip:],'g.',label='training loss')
plt.plot(epochs[skip:],val_loss[skip:],'b',label='validation loss')
plt.title('training and validation loss')
plt.title('epochs')
plt.title('loss')
plt.legend()
plt.show()


In [0]:
plt.clf()
mae=history_1.history['mae']
val_mae=history_1.history['val_mae']
plt.plot(epochs[skip:],mae[skip:],'g.',label='training mae')
plt.plot(epochs[skip:],val_mae[skip:],'b',label='validation loss mae')
plt.title('training and validation mean absolute error')
plt.xlabel('epochs')
plt.ylabel('mae')
plt.legend()
plt.show()

In [0]:
predictions=model_1.predict(x_train)
plt.clf()
plt.title('training data predicted vs actual values')
plt.plot(x_train,y_train,'b',label='actual')
plt.plot(x_train,predictions,'r',label='predicted')
plt.legend()
plt.show()


In [0]:
predictions=model_1.predict(x_test)
plt.clf()
plt.title('training data predicted vs actual values')
plt.plot(x_test,y_test,'b',label='actual')
plt.plot(x_test,predictions,'r',label='predicted')
plt.legend()
plt.show()


In [0]:
# from the graphs the model is not completely learn so we need to change our model
# so we need to increase the more layers

In [0]:
model_2=tf.keras.Sequential()
model_2.add(layers.Dense(16,activation='relu',input_shape=(1,)))
model_2.add(layers.Dense(8,activation='relu'))
model_2.add(layers.Dense(1))

In [0]:
model_2.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])

In [0]:
history_2=model_2.fit(x_train,y_train,epochs=600,batch_size=16,validation_data=(x_validate,y_validate))

In [0]:
loss=history_2.history['loss']
val_loss=history_2.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'b',label='training loss')
plt.plot(epochs,val_loss,'g',label='val_loss')
plt.title('training and val loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [0]:
skip=100
plt.clf()
plt.plot(epochs[skip:],loss[skip:],'b',label='training loss')
plt.plot(epochs[skip:],val_loss[skip:],'g',label='val loss')
plt.title('training and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [0]:
loss=model_2.evaluate(x_test,y_test)
predictions=model_2.predict(x_test)
plt.clf()
plt.title('comparision of actual and predictions')
plt.plot(x_test,y_test,'b',label='actual')
plt.plot(x_test,predictions,'r',label='predicted')
plt.legend()
plt.show()

In [0]:
# convert the model into tensorflow lite
#model_2.save('model_2.h5')
converter=tf.lite.TFLiteConverter.from_keras_model_file(model_2)
tflite_model=converter.convert()

# save the model to disk
open('sine_model.tflite','wb').write(tflite_model)

# convert the model to the tensorflow lite format with quantization
converter=tf.lite.TFLiteConverter.from_keras_model_file(model_2)
converter.optimizations=[tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model=converter.convert()

# save the model to the disk
open('sine_model_quantized.tflite','wb').write(tflite_model)


In [0]:
sine_model=tf.lite.Interpreter('sine_model.tflite')
sine_model_quantized=tf.lite.Interpreter('sine_model_quantized.tflite')
sine_model.allocate_tensors()
sine_model_quantized.allocate_tensors()

In [0]:
# get the input and output tensors so we can feed in values and get the results
sine_model_input=sine_model.tensor(sine_model.get_input_details()[0]['index'])
sine_model_output=sine_model.tensor(sine_model.get_output_details()[0]['index'])

# for quantization input and output tensors
sine_quantized_input=sine_model_quantized.tensor(sine_model_quantized.get_input_details()[0]['index'])
sine_quantized_output=sine_model_quantized.tensor(sine_model_quantized.get_output_details()[0]['index'])



In [0]:
# create the arrays to store the results
sine_model_predictions=np.empty(x_test.size)
sine_model_quantized_predictions=np.empty(x_test.size)

In [0]:
# Run each model's interpreter for each value and store the results in arrays
for i in range(x_test.size):
  sine_model_input().fill(x_test[i])
  sine_model.invoke()
  sine_model_predictions[i] = sine_model_output()[0]

  sine_model_quantized_input().fill(x_test[i])
  sine_model_quantized.invoke()
  sine_model_quantized_predictions[i] = sine_model_quantized_output()[0]

In [0]:
# See how they line up with the data
plt.clf()
plt.title('Comparison of various models against actual values')
plt.plot(x_test, y_test, 'bo', label='Actual')
plt.plot(x_test, predictions, 'ro', label='Original predictions')
plt.plot(x_test, sine_model_predictions, 'bx', label='Lite predictions')
plt.plot(x_test, sine_model_quantized_predictions, 'gx', label='Lite quantized predictions')
plt.legend()
plt.show()

In [0]:
#We can print the difference in file size:
import os
basic_model_size = os.path.getsize("sine_model.tflite")
print("Basic model is %d bytes" % basic_model_size)
quantized_model_size = os.path.getsize("sine_model_quantized.tflite")
print("Quantized model is %d bytes" % quantized_model_size)
difference = basic_model_size - quantized_model_size
print("Difference is %d bytes" % difference)

In [0]:
!apt-get -qq install xxd

In [0]:
# Save the file as a C source file
!xxd -i sine_model_quantized.tflite > sine_model_quantized.cc

In [0]:
# Print the source file
!cat sine_model_quantized.cc